## Consolida todas as LDs em um único Dataframe:

In [30]:
import pandas as pd
import os
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [31]:
# Caminho base para a pasta contendo os arquivos Excel
base_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasLDs\\'

# Dataframe para armazenar todos os resultados
df_LD_final = pd.DataFrame()

# Contadores para estatísticas
total_documentos = 0
documentos_com_comentarios = 0
documentos_com_disciplina = 0
cont_files = 0

# Iterar sobre todos os arquivos Excel na pasta
for filename in os.listdir(base_path):
    file_path = os.path.join(base_path, filename)
    cont_files += 1
    print(f"Lendo arquivo: {file_path}")
    
    # Ler o arquivo Excel
    df = pd.read_excel(file_path, sheet_name="VDRL")

    # Pré-processamento dos dados
    df.columns = df.iloc[6]
    df = df.iloc[7:]

    df.rename(columns = {"CLIENT DOCUMENT NUMBER":"CLIENT_DOCUMENT",  
                            "DOCUMENT TITLE":"DOCUMENT_TITLE",
                            "Disciplina":"Disciplina",
                            "Comments":"Comments"}, inplace = True)

    colunas_para_manter = ["CLIENT_DOCUMENT", "DOCUMENT_TITLE", "Disciplina", "Comments"]
    df.drop(columns=[col for col in df.columns if col not in colunas_para_manter], inplace = True)
    df.reset_index(drop=True, inplace = True)

    # df = df.iloc[:, :-2]
    # df.info()

    # Atualizar estatísticas
    total_documentos += len(df)
    documentos_com_comentarios += df['Comments'].notna().sum()
    documentos_com_disciplina += df['Disciplina'].notna().sum()
    
    # Concatenar com o DataFrame final
    df_LD_final = pd.concat([df_LD_final, df], axis=0, ignore_index=True)

# Resetar o índice do DataFrame final
df_LD_final.reset_index(drop=True, inplace=True)

# Remover espaços em branco e converter para maiúsculas
#df_LD_final = df_LD_final.dropna(subset=['DOCUMENT_TITLE'])
#df_LD_final['DOCUMENT_TITLE'] = df_LD_final['DOCUMENT_TITLE'].apply(lambda x: x.strip().upper())
#df_LD_final['Disciplina'] = df_LD_final['Disciplina'].apply(lambda x: x.strip().upper() if pd.notna(x) else x)
#df_LD_final['Comments'] = df_LD_final['Comments'].apply(lambda x: x.strip().upper() if pd.notna(x) else x)

# Função para limpar e converter para maiúsculas
def clean_text(x):
    if isinstance(x, str) and pd.notna(x):
        return x.strip().upper()
    return x

# Aplicar a função nas colunas
df_LD_final = df_LD_final.dropna(subset=['DOCUMENT_TITLE'])
df_LD_final['DOCUMENT_TITLE'] = df_LD_final['DOCUMENT_TITLE'].apply(clean_text)
df_LD_final['Disciplina'] = df_LD_final['Disciplina'].apply(clean_text)
df_LD_final['Comments'] = df_LD_final['Comments'].apply(clean_text)

# Substituir valores que contém "YES" por "YES" somente e as que não contém "YES" por ""
# Definindo a função para aplicar na coluna
def convert_yes(value):
    if isinstance(value, str):
        value_upper = value.upper().strip()
        if 'YES' in value_upper or value_upper == 'Y' or value_upper == 'X' or value_upper == 'SIM':
            return 'YES'
    return ''

# Criando uma nova coluna 'Comments_new' usando map
df_LD_final['Comments_new'] = df_LD_final['Comments'].map(convert_yes)

# Atualizar estatísticas
total_documentos = len(df_LD_final)
documentos_com_comentarios = df_LD_final['Comments_new'].value_counts()['YES']
documentos_com_disciplina = df_LD_final['Disciplina'].count()

# Calcular estatísticas finais
if total_documentos > 0:
    porcentagem_com_comentarios = (documentos_com_comentarios / total_documentos) * 100
    porcentagem_com_disciplina = (documentos_com_disciplina / total_documentos) * 100
else:
    porcentagem_com_comentarios = 0
    porcentagem_com_disciplina = 0

# Criar DataFrame de estatísticas
df_estatisticas = pd.DataFrame({
    'Métrica': ['Total de Documentos', 'Documentos com YES', 'Documentos com Disciplina',
                '% com Comentários', '% com Disciplina'],
    'Valor': [total_documentos, documentos_com_comentarios, documentos_com_disciplina,
              f"{porcentagem_com_comentarios:.2f}%", f"{porcentagem_com_disciplina:.2f}%"]
})

# Salvar o DataFrame final e as estatísticas
output_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_comment_combined.xlsx'

with pd.ExcelWriter(output_file) as writer:
    df_LD_final.to_excel(writer, sheet_name='LD', index=False)
    df_estatisticas.to_excel(writer, sheet_name='Estatísticas', index=False)

print(f'Arquivo {output_file} salvo em: {Path(output_file).resolve()}')

# Imprimir estatísticas no console
print("\nMétricas:")
print(f"Número de arquivos processados: {cont_files}")
print(f"Total de Documentos: {total_documentos}")
print(f"Documentos com YES: {documentos_com_comentarios} ({porcentagem_com_comentarios:.2f}%)")
print(f"Documentos com Disciplina: {documentos_com_disciplina} ({porcentagem_com_disciplina:.2f}%)")


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-210-YA7-001_RISER HARD PIPE.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-EP8-096_0_FLOW METERING SYSTEM - rev1.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HG8-001_PRESSURE VACUUM RELIEF VALVE.xlsx


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HM9-001_0_PRESSURE VESSELS_TS.xlsx


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HYE-001_NON_METALLIC_TANKS.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-J7B-001_Manual Valve JIANGSU.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-KFQ-001_0_comments.xlsx


...

Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-5511-76F-A2L-001_HELIDICK LIGHT SYSTEM.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-5511-76F-OD7-001_NAVIGATION AID SYSTEM.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-5520-800-AK1-001_0 - CSS TOPSIDE.xlsx


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-5522-855-EQ9-001_ADDRESSABLE FIRE DETECTION SYSTEM.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-5522-855-S4N-001_F&G DETECTOR.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-5525-911-HHG-001_GAS SAMPLING SYSTEM.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-552B-900-S17-001_MMS (Machinery Monitoring System).xlsx


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-6440-940-NH9-001_0_HOSE REELS.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-8220-600-VST-001_WORKSHOP EQUIPMENT.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-8222-91A-S4Q-001_LABORATORY.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-8268-622-G8U-001_Elevator.xlsx
Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\LD_comment_combined.xlsx salvo em: C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\LD_comment_combined.xlsx

Métricas:
Número de arquivos processados: 143
Total de Documentos: 15786
Documentos com YES: 7883 (49.94%)
Documentos com Disciplina: 8371 (53.03%)


### Avalia com quem está a VDA sem comentário

In [32]:
import pandas as pd
import os

# Definir os caminhos dos arquivos
vda_path = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\TodasVDAs\VDA 25.05.23.xlsx'
ld_path = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\LD_comment_combined.xlsx'

# Carregar os arquivos Excel
vda_df = pd.read_excel(vda_path, sheet_name="DADOS")
ld_df = pd.read_excel(ld_path, sheet_name="LD")

# Criar as novas colunas no DataFrame VDA
vda_df['Disciplina'] = ''
vda_df['Comments_new'] = ''

# Iterar sobre as linhas do DataFrame VDA
for index, row in vda_df.iterrows():
    ref_doc = row['REFERENCE DOCUMENT NAME']
    
    # Procurar o documento correspondente no DataFrame LD
    matching_row = ld_df[ld_df['CLIENT_DOCUMENT'] == ref_doc]
    
    if not matching_row.empty:
        # Se encontrou uma correspondência
        vda_df.at[index, 'Disciplina'] = matching_row['Disciplina'].values[0]
        
        if pd.notna(matching_row['Comments_new'].values[0]):
            vda_df.at[index, 'Comments_new'] = matching_row['Comments_new'].values[0]
        else:
            vda_df.at[index, 'Comments_new'] = 'NO'
    else:
        # Se não encontrou correspondência
        vda_df.at[index, 'Comments_new'] = 'Documento não encontrado na LD_comment_combined.xlsx'

# Remover linhas duplicadas com base na coluna "VDA NAME"
if "VDA NAME" in vda_df.columns and "VDA VERSION" in vda_df.columns:
    print(f"Número de linhas antes da remoção de duplicatas: {len(vda_df)}")
    # Sort by VDA VERSION in descending order and keep first occurrence of each VDA NAME
    vda_df = vda_df.sort_values('VDA VERSION', ascending=False).drop_duplicates(subset='VDA NAME', keep='first')
    print(f"Número de linhas após a remoção de duplicatas: {len(vda_df)}")
else:
    print("Aviso: Colunas necessárias não encontradas. Não foi possível remover duplicatas.")

# Salvar o DataFrame VDA atualizado
output_path = os.path.join(os.path.dirname(vda_path), 'VDA 25.05.23_updated.xlsx')
vda_df.to_excel(output_path, index=False)

print(f"Arquivo atualizado salvo em: {output_path}")


Número de linhas antes da remoção de duplicatas: 668
Número de linhas após a remoção de duplicatas: 665
Arquivo atualizado salvo em: C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\TodasVDAs\VDA 25.05.23_updated.xlsx


### Avalia a quantidade de cometários solicitados para comentar versus os realizados

In [33]:
import pandas as pd
import os
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Carregar os arquivos
input_file1 = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_comment_combined.xlsx'
df_LD_final = pd.read_excel(input_file1, sheet_name="LD")
input_file2 = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasVDAs\\VDA 25.05.23.xlsx'
vendor_docs = pd.read_excel(input_file2, sheet_name="DADOS")

# Adicionar novas colunas ao df_LD_final
df_LD_final['VDA'] = ''
df_LD_final['Version'] = ''
df_LD_final['Status'] = ''
df_LD_final['Workflow Status'] = ''

# Atualizar df_LD_final com informações do vendor_docs
for index, row in df_LD_final.iterrows():
    matching_row = vendor_docs[vendor_docs['REFERENCE DOCUMENT NAME'] == row['CLIENT_DOCUMENT']]
    if not matching_row.empty:
        df_LD_final.at[index, 'VDA'] = matching_row['VDA NAME'].values[0]
        df_LD_final.at[index, 'Version'] = matching_row['VDA VERSION'].values[0]
        df_LD_final.at[index, 'Status'] = matching_row['VDA RELEASE STATUS / VOTE'].values[0]
        df_LD_final.at[index, 'Workflow Status'] = matching_row['VDA WORKFLOW STATUS'].values[0]

# Filtrar apenas os documentos com VDA preenchido
df_LD_final_with_vda = df_LD_final[df_LD_final['VDA'].notna() & (df_LD_final['VDA'] != '')]

# Calcular estatísticas apenas para documentos com VDA preenchido
comments_yes = len(df_LD_final_with_vda[df_LD_final_with_vda['Comments_new'] == 'YES'])
comments_added = len(df_LD_final_with_vda[df_LD_final_with_vda['Status'] == 'COMMENTS ADDED'])
total_docs_with_vda = len(df_LD_final_with_vda)

# Criar DataFrame de estatísticas
stats_df = pd.DataFrame({
    'Métrica': ['Total de documentos com VDA preenchido',
                'Documentos com YES em Comments_new',
                'Documentos com COMMENTS ADDED em Status'],
    'Quantidade': [total_docs_with_vda, comments_yes, comments_added]
})

# Salvar resultados
with pd.ExcelWriter('C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_comment_combined.xlsx') as writer:
    df_LD_final.to_excel(writer, sheet_name='LD', index=False)
    stats_df.to_excel(writer, sheet_name='Estatísticas Comentários', index=False)

print(f"\nEstatísticas (apenas para documentos com VDA preenchido):")
print(f"Total de documentos com VDA preenchido: {total_docs_with_vda}")
print(f"Documentos com YES em Comments_new: {comments_yes}")
print(f"Documentos com COMMENTS ADDED em Status: {comments_added}")



Estatísticas (apenas para documentos com VDA preenchido):
Total de documentos com VDA preenchido: 551
Documentos com YES em Comments_new: 481
Documentos com COMMENTS ADDED em Status: 371


### Avalia dos documentos emitidos com VDA quais foram solicitados para comentar
- inverso da lógica anterior

In [34]:

import pandas as pd
import os
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Carregar os arquivos
input_file1 = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_comment_combined.xlsx'
df_LD_final = pd.read_excel(input_file1, sheet_name="LD")

input_file2 = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasVDAs\\VDA 25.05.23.xlsx'
vendor_docs = pd.read_excel(input_file2, sheet_name="DADOS")

output_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasVDAs\\Vendor docs 25.05.23_updated.xlsx'

# Atualizar vendor_docs com informações do df_LD_final
for index, row in vendor_docs.iterrows():
    matching_row = df_LD_final[df_LD_final['CLIENT_DOCUMENT'] == row['REFERENCE DOCUMENT NAME']]
    if not matching_row.empty:
        vendor_docs.at[index, 'Disciplina'] = matching_row['Disciplina'].values[0]
        vendor_docs.at[index, 'Comments'] = matching_row['Comments'].values[0]
        vendor_docs.at[index, 'Comments_new'] = matching_row['Comments_new'].values[0]

# Calcular estatísticas
comments_yes = len(vendor_docs[vendor_docs['Comments_new'] == 'YES'])
comments_added = len(vendor_docs[vendor_docs['REFERENCE DOCUMENT RELEASE STATUS / VOTE'] == 'COMMENTS ADDED'])
total_docs = len(vendor_docs)

# Criar DataFrame de estatísticas
stats_df = pd.DataFrame({
    'Métrica': ['Total de documentos',
                'Documentos com YES em Comments_new',
                'Documentos com COMMENTS ADDED em Status'],
    'Quantidade': [total_docs, comments_yes, comments_added]
})

# Salvar resultados
print(f"\nArquivo Salvo em {output_file}")
with pd.ExcelWriter(output_file) as writer:
    vendor_docs.to_excel(writer, sheet_name='Sheet1', index=False)
    stats_df.to_excel(writer, sheet_name='Estatísticas Comentários', index=False)

print(f"\nEstatísticas:")
print(f"Total de documentos: {total_docs}")
print(f"Documentos com YES em Comments_new: {comments_yes}")
print(f"Documentos com COMMENTS ADDED em Status: {comments_added}")



Arquivo Salvo em C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasVDAs\Vendor docs 25.05.23_updated.xlsx

Estatísticas:
Total de documentos: 668
Documentos com YES em Comments_new: 460
Documentos com COMMENTS ADDED em Status: 463
